In [2]:
import pandas as pd
import string
import re

In [3]:
df = pd.read_excel(r'E:\Study\Analytix Labs\Projects\project\5. Bank Reviews-Complaints Analysis\BankReviews.xlsx')

In [4]:
data = pd.read_excel(r'E:\Study\Analytix Labs\Projects\project\5. Bank Reviews-Complaints Analysis\BankReviews.xlsx')

In [5]:
df = df.drop_duplicates(['Stars','Reviews','BankName'])

In [6]:
df.head()

,Date,Stars,Reviews,BankName
0,2017-04-10,5,"Great job, Wyndham Capital! Each person was pr...",Wyndham Capital Mortgage
1,2017-02-10,5,Matthew Richardson is professional and helpful...,Wyndham Capital Mortgage
2,2017-08-21,5,We had a past experience with Wyndham Mortgage...,Wyndham Capital Mortgage
3,2017-12-17,5,We have been dealing with Brad Thomka from the...,Wyndham Capital Mortgage
4,2016-05-27,5,I can't express how grateful I am for the supp...,Wyndham Capital Mortgage


In [7]:
df.shape

(398, 4)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 398 entries, 0 to 490
Data columns (total 4 columns):
Date        398 non-null datetime64[ns]
Stars       398 non-null int64
Reviews     398 non-null object
BankName    398 non-null object
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 15.5+ KB


# Removing punctuations , stopwords , newline characters and numbers from the data.

In [10]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [11]:
df.Reviews = df.Reviews.dropna().apply(lambda x : "".join([c for c in x if c not in string.punctuation]))
df.Reviews = df.Reviews.apply(lambda x : x.lower())

In [12]:
df.Reviews = df.Reviews.apply(lambda x: x.strip().replace("  ", ' ').replace('\r', ' ').replace('\n', ' ').replace('"', ''))

In [13]:
df.Reviews = df.Reviews.apply(lambda x: re.sub(r'[0-9]+|\S+[0-9]\S+\ss',"",x))

In [14]:
df.Reviews[150]

'my loan officer was a really big help he was very patient and helped me save some money on the closing cost thanks again i would definitely refer to a friend'

In [15]:
import nltk
#from nltk.tokenize import TreebankWordTokenizer
#tokenizer = TreebankWordTokenizer()
df.Reviews = df.Reviews.apply(lambda x : nltk.word_tokenize(x))
df.Reviews[1]

In [17]:
from nltk.corpus import stopwords
df.Reviews = df.Reviews.apply(lambda x :[y for y in x if y not in set(stopwords.words('english'))])

In [18]:
df.Reviews[1]

['matthew',
 'richardson',
 'professional',
 'helpful',
 'helped',
 'us',
 'find',
 'correct',
 'product',
 'mortgage',
 'thank',
 'much',
 'excellent',
 'service',
 'matthew']

In [5]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from collections import Counter

In [20]:
def get_pos( word ):
    w_synsets = wordnet.synsets(word)

    pos_counts = Counter()
    pos_counts["n"] = len(  [ item for item in w_synsets if item.pos()=="n"]  )
    pos_counts["v"] = len(  [ item for item in w_synsets if item.pos()=="v"]  )
    pos_counts["a"] = len(  [ item for item in w_synsets if item.pos()=="a"]  )
    pos_counts["r"] = len(  [ item for item in w_synsets if item.pos()=="r"]  )
    
    most_common_pos_list = pos_counts.most_common(3)
    return most_common_pos_list[0][0]

In [13]:
wordnet.synsets('helped').pos()

AttributeError: 'list' object has no attribute 'pos'

In [14]:
wnl = WordNetLemmatizer()
df.Reviews = df.Reviews.apply(lambda x: [wnl.lemmatize(word, get_pos(word)) for word in x])


NameError: name 'df' is not defined

In [22]:
df.Reviews[1]

['matthew',
 'richardson',
 'professional',
 'helpful',
 'help',
 'u',
 'find',
 'correct',
 'product',
 'mortgage',
 'thank',
 'much',
 'excellent',
 'service',
 'matthew']

In [23]:
bgm_Reviews = df.Reviews.apply(lambda x :list(nltk.bigrams(x)))

In [24]:
df.Reviews = df.Reviews.apply(lambda x: ' '.join([y for y in x]))

In [25]:
df.Reviews

0      great job wyndham capital person professional ...
1      matthew richardson professional helpful help u...
2      past experience wyndham mortgage would without...
3      deal brad thomka begin start stressful time u ...
4      cant express grateful support zach provide fam...
5      pleasure work wyndham capital september family...
6      experience mattison beyond greatly professiona...
7      patrick answer question email immediately spen...
8      love work group people laugh phone always answ...
9      great web interface loan application document ...
10     work michelle wyndham go really well first tim...
11     usten butler bring humanity connection stiff l...
12     jay easy get hold question huge there lot whol...
13     million friend would recommend wyndham capital...
14     chaz fantastic throughout entire lend process ...
15     austen awsome every step way refi new home pur...
16     salesperson keep push cash refi even tell im l...
17     worst experience ever li

In [28]:
from textblob import TextBlob

In [29]:
def get_bank_sentiment(review): 
    
    # create TextBlob object of passed  text 
    analysis = TextBlob(review) 
    # set sentiment 
    if analysis.sentiment.polarity > 0: 
        return 'positive'
    elif analysis.sentiment.polarity == 0: 
        return 'neutral'
    else: 
        return 'negative'

In [30]:
df['Sentiment'] = df.Reviews.apply(lambda x: get_bank_sentiment(x))

In [31]:
df[(df.Sentiment== 'negative') & (df.Stars == 5)]

,Date,Stars,Reviews,BankName,Sentiment
16,2016-10-07,5,salesperson keep push cash refi even tell im l...,Wyndham Capital Mortgage,negative
59,2017-07-22,5,new federal regulation make tedious time consu...,Reliance First Capital,negative
88,2017-03-11,5,wesley white work tirelessly patiently get loa...,Reliance First Capital,negative
148,2016-02-14,5,slight difficulty online document sign,Pacific Beneficial Mortgage Co,negative
214,2017-07-07,5,hesitant use nonlocal mortgage company loan of...,North American Savings Bank,negative


In [35]:
len(df[df.Sentiment == 'positive'])*100/len(df)

86.93467336683418

In [36]:
len(df[df.Sentiment == 'negative'])*100/len(df)

9.045226130653266

In [37]:
len(df[df.Sentiment == 'neutral'])*100/len(df)

4.0201005025125625

In [32]:
df_neg = df[df.Sentiment == 'negative']

In [33]:
df_neg.shape

(36, 5)

In [39]:
# bag of words
from sklearn.feature_extraction.text import CountVectorizer
def bow_extractor(corpus):
    vectorizer = CountVectorizer()
    features   = vectorizer.fit_transform(corpus)
    return  vectorizer ,features 

bow_vectorizer , bow_features = bow_extractor(df_neg.Reviews)
features = bow_features.todense()
print(features)


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


In [40]:
feature_names =  bow_vectorizer.get_feature_names()


In [41]:
def display_features(features,features_names):
    data = pd.DataFrame(data=features,columns=feature_names)
    return(data)
    
display_features(features,feature_names)


,able,absolutely,accept,accurate,across,action,adam,addedconsolidationit,additional,advertise,...,worth,would,wouldnt,wrong,wyndum,year,yes,yet,youi,zero
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
def tfidf_extractor(corpus):
    vectorizer = TfidfVectorizer(min_df = 1, use_idf = True,norm = 'l2')
    features   = vectorizer.fit_transform(corpus)
    return vectorizer , features


In [47]:
tfidf_vectorizer ,tfidf_features =  tfidf_extractor(df_neg.Reviews)
display_features(np.round(tfidf_features.todense(),2),feature_names)

,able,absolutely,accept,accurate,across,action,adam,addedconsolidationit,additional,advertise,...,worth,would,wouldnt,wrong,wyndum,year,yes,yet,youi,zero
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.13,0.00,0.00,0.15,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.23,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.07,0.00,0.16,0.00,0.00,0.00,0.00,0.00,0.00
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [50]:
data1 = display_features(np.round(tfidf_features.todense(),2),feature_names)

In [51]:
from sklearn.decomposition import LatentDirichletAllocation

In [66]:
total_topics = 3
lda = LatentDirichletAllocation(n_topics=total_topics,max_iter=100,learning_method='online',learning_offset=50,random_state=42)

In [67]:
lda.fit(data1)

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50,
             max_doc_update_iter=100, max_iter=100, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=3, perp_tol=0.1,
             random_state=42, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [68]:
weights = lda.components_

In [69]:
def get_topics_terms_weights(weights, feature_names):
    feature_names = np.array(feature_names)
    sorted_indices = np.array([list(row[::-1]) for row in np.argsort(np.abs(weights))])
    sorted_weights = np.array([list(wt[index]) for wt, index  in zip(weights, sorted_indices)])
    sorted_terms = np.array([list(feature_names[row]) for row in sorted_indices])
    
    topics = [np.vstack((terms.T, term_weights.T)).T for terms, term_weights in zip (sorted_terms, sorted_weights)]
    return topics

In [70]:
def print_topics_udf(topics, total_topics= 1, weight_threshold = 0.0001, display_weights = False, num_terms = None):
    for index in range(total_topics):
        topic = topics[index]
        topic = [(term, float(wt)) for term, wt in topic]
        topic = [(word, round(wt,2)) for word, wt in topic if abs(wt) >= weight_threshold]
        
        if display_weights:
            print('Topic #'+ str(index + 1)+ ' with weights')
            print(topic[:num_terms] if num_terms else topic)
        else:
            print('Topic #'+ str(index + 1)+ ' without weights')
            tw = [term for term, wt in topic]
            print(tw[:num_terms] if num_terms else tw)

In [71]:
topics = get_topics_terms_weights(weights, feature_names)

In [72]:
print_topics_udf(topics = topics, total_topics = total_topics, num_terms = 15, display_weights = True)

Topic #1 with weights
[('unprofessional', 1.39), ('previous', 1.28), ('would', 1.26), ('number', 1.19), ('list', 1.13), ('contact', 1.12), ('close', 1.12), ('phone', 1.12), ('recommend', 1.1), ('site', 1.05), ('register', 1.05), ('disturb', 1.05), ('lender', 0.97), ('company', 0.97), ('appraisal', 0.96)]
Topic #2 with weights
[('information', 1.16), ('everything', 1.11), ('call', 1.09), ('much', 1.07), ('send', 0.97), ('home', 0.96), ('would', 0.95), ('new', 0.95), ('get', 0.94), ('say', 0.9), ('accept', 0.89), ('wouldnt', 0.89), ('screw', 0.87), ('purchasing', 0.87), ('miserable', 0.87)]
Topic #3 with weights
[('bad', 1.5), ('loan', 1.37), ('process', 1.31), ('close', 1.24), ('get', 1.14), ('make', 1.04), ('week', 1.01), ('broker', 0.99), ('go', 0.94), ('work', 0.91), ('call', 0.9), ('seem', 0.9), ('experience', 0.88), ('mortgage', 0.85), ('use', 0.84)]


In [ ]:
# Topic interpretation
#1. Unprofessional staff
#2. Misguidance or hidden policies
#3. Sluggish Processing

In [74]:
bow_vectorizer , bow_features = bow_extractor(df.Reviews)
features = bow_features.todense()
print(features)


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [75]:
feature_names =  bow_vectorizer.get_feature_names()
display_features(features,feature_names)


,aaron,ability,able,absoltely,absolute,absolutely,accept,acceptable,access,accessible,...,xx,year,yelp,yep,yes,yet,youi,zach,zelada,zero
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
tfidf_vectorizer ,tfidf_features =  tfidf_extractor(df.Reviews)
data1 = display_features(np.round(tfidf_features.todense(),2),feature_names)

In [78]:
x = data1

In [79]:
y = df.Stars

In [80]:
from sklearn.model_selection import train_test_split
from sklearn import svm

In [81]:
X_train,X_test,Y_train,Y_test = train_test_split(x,y,random_state=123,test_size=0.3)

In [82]:
clf = svm.SVC(kernel='linear', C = 1.0)
clf.fit(X_train,Y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [83]:
y_pred = clf.predict(X_test)

In [84]:
from sklearn import metrics
print(metrics.accuracy_score(Y_test, y_pred))

0.9333333333333333


In [85]:
print(metrics.confusion_matrix(Y_test, y_pred))

[[18  6]
 [ 2 94]]


In [86]:
print(metrics.roc_auc_score(Y_test, y_pred))

0.8645833333333334


NameError: name 'metrics' is not defined